In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector

Сгенерируем данные для работы

In [178]:
x_data_generated, y_data_generated = make_classification(scale=1)

Вычислим среднюю точность предсказаний модели Логистической регрессии

In [179]:
begin_data = cross_val_score(LogisticRegression(), x_data_generated, y_data_generated, scoring='accuracy').mean()
begin_data

0.93

Проведем отбор признаков с помощью корреляционной матрицы

In [180]:
corr = pd.DataFrame(x_data_generated)

In [181]:
corr.corr().style.background_gradient(cmap='coolwarm')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.049180,-0.023663,0.019875,-0.015526,0.046464,0.852845,-0.005127,-0.303868,-0.102080,-0.068993,0.103040,0.114104,-0.020665,0.092933,-0.720107,-0.134842,0.054654,0.128852,-0.079372
1,0.049180,1.000000,-0.107164,-0.111502,0.020263,0.044630,0.049988,0.154559,-0.000266,0.012227,-0.022746,0.194172,-0.115136,0.062201,-0.087436,-0.024725,-0.024029,-0.159862,0.017118,0.091717
2,-0.023663,-0.107164,1.000000,0.120529,0.030282,0.093385,-0.160227,0.008673,-0.248331,0.035227,0.025882,-0.054086,-0.119628,-0.118246,-0.080299,-0.169057,0.014279,0.135384,0.072617,-0.005599
3,0.019875,-0.111502,0.120529,1.000000,0.035022,0.005581,-0.016617,0.007563,-0.067284,0.037534,0.034484,-0.116323,-0.013357,-0.072385,-0.077061,-0.058916,0.039412,0.110831,-0.049594,0.013666
4,-0.015526,0.020263,0.030282,0.035022,1.000000,-0.148115,-0.049633,-0.085832,-0.061680,0.059764,0.060269,0.236019,0.043153,0.128818,-0.150009,-0.037177,0.182841,0.137452,-0.034426,0.001822
5,0.046464,0.044630,0.093385,0.005581,-0.148115,1.000000,0.077794,-0.038968,0.055518,0.107769,-0.092098,-0.084465,-0.066112,0.020003,0.044424,0.017258,-0.074526,-0.117869,0.127239,0.182834
6,0.852845,0.049988,-0.160227,-0.016617,-0.049633,0.077794,1.000000,-0.031615,0.238321,-0.096921,-0.116847,0.174219,0.032497,-0.006334,0.072311,-0.251829,-0.117184,0.095548,0.083330,-0.118621
7,-0.005127,0.154559,0.008673,0.007563,-0.085832,-0.038968,-0.031615,1.000000,-0.048147,-0.124659,-0.005232,0.045462,-0.109457,0.068698,-0.121709,-0.032509,0.115426,-0.176537,-0.151968,-0.036624
8,-0.303868,-0.000266,-0.248331,-0.067284,-0.061680,0.055518,0.238321,-0.048147,1.000000,0.013024,-0.084871,0.126225,-0.152932,0.026878,-0.040914,0.879871,0.036988,0.072679,-0.087616,-0.068804
9,-0.102080,0.012227,0.035227,0.037534,0.059764,0.107769,-0.096921,-0.124659,0.013024,1.000000,0.172905,0.177266,-0.056193,0.111912,0.097631,0.060403,-0.109220,-0.126392,-0.071218,-0.016929


Признаки 11, 5   и 12 4 имеют сильную корреляцию, удалим их

In [182]:
corr = np.delete(x_data_generated, [6,8],1)

Вычислим среднюю точность предсказаний после отбора на корреляционной матрице

In [183]:
corr_data = cross_val_score(RandomForestClassifier(), corr, y_data_generated, scoring='accuracy').mean()
corr_data

0.95

Сделаем отбор признаков с помощью VarianceThreshold

In [184]:
selector = VarianceThreshold(threshold=1)
select_feach = selector.fit_transform(x_data_generated)
select_feach.shape

(100, 10)

Вычислим среднюю точность предсказаний после отбора на VarianceThreshold

In [185]:
var_threshold = cross_val_score(LogisticRegression(), select_feach, y_data_generated, scoring='accuracy').mean()
var_threshold

0.9199999999999999

Сделаем отбор признаков с помощью скоринговой функции для классификации SelectKBest

In [186]:
select_kbest = SelectKBest(k=5).fit_transform(x_data_generated, y_data_generated)
select_kbest.shape

(100, 5)

Вычислим среднюю точность предсказаний модели Логистической регрессии

In [187]:
kbest = cross_val_score(LogisticRegression(), select_kbest, y_data_generated, scoring='accuracy').mean()
kbest

0.97

Отбор признаков с помощью логистической регрессии

In [188]:
selector_model = SelectFromModel(estimator=LogisticRegression(C=1, penalty='l2')).fit_transform(x_data_generated, y_data_generated)

In [189]:
select_from_model = cross_val_score(LogisticRegression(), selector_model, y_data_generated, scoring='accuracy').mean()
select_from_model

0.96

Отбор 5 наиболее важных признаков с помощью RandomForest

In [190]:
columns = pd.DataFrame(x_data_generated).columns.tolist()
columns

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [191]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(x_data_generated,y_data_generated)
feature_imp = pd.Series(clf.feature_importances_,index=columns).sort_values(ascending=False)
col_feature_imp = list(feature_imp.index[:5])

In [192]:
feature_importance = x_data_generated[:, col_feature_imp]

In [193]:
random_forest = cross_val_score(LogisticRegression(), feature_importance, y_data_generated, scoring='accuracy').mean()
random_forest

0.96

Произведем  перебор признаков с помощью SequentialFeatureSelector.

In [194]:
sequential_feature = SequentialFeatureSelector(estimator=LogisticRegression(), n_features_to_select=7).fit_transform(x_data_generated, y_data_generated)

In [195]:
seq_feature_sel = cross_val_score(LogisticRegression(), sequential_feature, y_data_generated, scoring='accuracy').mean()
seq_feature_sel

0.9800000000000001

In [198]:
temp = {'begin_data': [begin_data], 'corr': [corr_data], 'var_threshold': [var_threshold], 
     'kbest': [kbest], 'select_from_model': [select_from_model], 'random_forest': [random_forest],
    'seq_feature_sel': [seq_feature_sel] }
result_tab = pd.DataFrame(temp)
result_tab

,begin_data,corr,var_threshold,kbest,select_from_model,random_forest,seq_feature_sel
0,0.93,0.95,0.92,0.97,0.96,0.96,0.98


### Вывод: 
По итогам отбора признаков различными методами, наиболее значимые результаты показали методы Kbest и SequentialFeatureSelector